### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [422]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import time

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


First five records in our dataframe.

## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [423]:
#a)
baby_df.review = baby_df.review.apply(lambda x: remove_punctuation((str(x))))

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

Row that was with punctuations now is same as string with same sentence and without punctuations, so it works properly.

In [424]:
#b)

baby_df = baby_df.fillna("")

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [425]:
#c)
baby_df = baby_df.drop(baby_df[baby_df.rating == 3].index)
#short test:
sum(baby_df["rating"] == 3)

0

Sum of ratings which equals 3 is 0.

In [426]:
#d) 
def set_val(rating):
    if rating <= 2:
        return -1
    if rating >= 4:
        return 1

baby_df.rating = baby_df.rating.apply(lambda x: set_val(x))
#short test:
sum(baby_df["rating"]**2 != 1)

0

Sum of ratings different than 1 is 0, there are no negative ratings so no ratings where added, thus function works properly.

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [427]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [428]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(vectorizer.get_feature_names())
print(X_test_example.todense())

['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [429]:
#a)
start = time.time()

review_train, review_test, rating_train, rating_test = train_test_split(baby_df.review,baby_df.rating, test_size=0.3, random_state=45)


Splitting dataset into training and test sets in ratio 7 to 3.

In [430]:
#b)
vectorizer = CountVectorizer()
vectorizer.fit(baby_df.review)

review_train_V = vectorizer.transform(review_train)
review_test_V = vectorizer.transform(review_test)

Fitting vectorizer to review and then do the data transformation.

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [431]:
#a)
model = LogisticRegression()
model.fit(review_train_V, rating_train)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Creating model of logistic regression and fitting data.

In [432]:
#b)
coefs = model.coef_[0]
d = {"Coef": coefs, "Words": vectorizer.get_feature_names()}
words_table = pd.DataFrame(d)

#hint: model.coef_, vectorizer.get_feature_names()

Make a dataframe with coefficients and words.

In [433]:
words_table.sort_values(by="Coef", ascending=False).head(10)

,Coef,Words
15420,2.396217,awesome
46753,2.311483,excellent
93841,2.298750,pleased
55107,2.120888,glad
60684,2.074559,highly
11747,2.054904,amazing
139488,2.047722,worry
106408,1.880788,satisfied
31732,1.797631,complaint
46671,1.777552,exactly


Ten most positive words.

In [434]:
words_table.sort_values(by="Coef").head(10)

,Coef,Words
139538,-3.184703,worst
39961,-2.981218,disappointing
32052,-2.602091,concept
132783,-2.563993,useless
61887,-2.510443,horrible
122776,-2.472915,terrible
94612,-2.435888,poor
94623,-2.413184,poorly
103792,-2.271653,returning
103769,-1.975957,returned


Ten most negative words.

As we can see not all the words are classified correctly, for egzample word "worry" in positive words.

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [435]:
#a)
rating_pred = model.predict(review_test_V) # calculate ratings based on review using model creating by LogisticRegression

Calculate ratings based on review using LogisticRegression model.

In [436]:
#b)
rating_pred_proba = model.predict_proba(review_test_V)

#hint: model.predict_proba()

In [437]:
#c) 
indexes_P = np.argsort(rating_pred_proba[:,1])[-5:]
print("Positive:\n", review_test.iloc[indexes_P])

indexes_N = np.argsort(rating_pred_proba[:,0])[-5:]
print("\nNegative:\n", review_test.iloc[indexes_N])

#hint: use the results of b)

Positive:
 116083    Ive had this stroller for a little more than s...
21557     Ok I read all the reviews already posted here ...
114796    My husband and I cannot state enough how much ...
180953    AMAZING stroller  It took me about 2 minutes t...
144112    Background Ive been using Grovia diapers for f...
Name: review, dtype: object

Negative:
 54418    THIS BASSINET IS OVERPRICED AND RIDICULOUS  If...
16042    We have not had ANY luck with FisherPrice prod...
59062    WARNING to all owners newer model Peg Perego S...
87026    First off I did manage to find this product fo...
10180    Please see my email to the companyHelloI am wr...
Name: review, dtype: object


In [438]:
#d) 
acc = sum(rating_pred==rating_test)/len(rating_test)
print("Accuracy of predictions:", acc)
time1 = time.time() - start

Accuracy of predictions: 0.9308159756926399


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [439]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [440]:
#a)
start2= time.time()

X_trainSW, X_testSW, y_trainSW, y_testSW = train_test_split(baby_df.review,baby_df.rating, test_size=0.3, random_state=44)


In [441]:
#b)

vectorizer = CountVectorizer()
vectorizer.fit(significant_words)

X_trainSW_V = vectorizer.transform(X_trainSW.values)
X_testSW_V = vectorizer.transform(X_testSW.values)

model = LogisticRegression()
model.fit(X_trainSW_V, y_trainSW)

coefs = model.coef_[0]
d = {"Coef": coefs, "Names": vectorizer.get_feature_names()}
words_table = pd.DataFrame(d)

y_predSW = model.predict(X_testSW_V)
y_pred_probaSW = model.predict_proba(X_testSW_V)

indexes_P = np .argsort(y_pred_probaSW[:,1])[-5:]
print("Positive:\n", X_testSW.iloc[indexes_P])

indexes_N = np.argsort(y_pred_probaSW[:,0])[-5:]
print("\nNegative:\n", X_testSW.iloc[indexes_N])

acc2 = sum(y_predSW==y_testSW)/len(y_testSW)
print("\nAccuracy of predictions:", acc2)
time2 = time.time() - start2

Positive:
 90795     This diaper bag is PERFECT There are a lot of ...
181875    There is probably nothing more scrutinized the...
174637    Over the last five years I have tried many dif...
103297    I was very excited when I heard Chicco was fin...
12035     My daughter 8 months old loves this toy and ha...
Name: review, dtype: object

Negative:
 56798     I was excited to give these instruments to my ...
35763     Day 1 Assembled it Had it up and running playi...
140418    I am a researchaholic in general and have rese...
119175    Babyhaven knew that the items were never taken...
22491     My wife has been sucessful using the pump but ...
Name: review, dtype: object

Accuracy of predictions: 0.8682285211689921


In [442]:
#c)

print("Time without limited dictionary ", time1)
print("Time with limited dictionary ", time2)
print("Accuracy of predictions without limited dictionary ", acc)
print("Accuracy of predictions with limited dictionary ", acc2)

#hint: %time, %timeit

Time without limited dictionary  53.57116198539734
Time with limited dictionary  15.101290941238403
Accuracy of predictions without limited dictionary  0.9308159756926399
Accuracy of predictions with limited dictionary  0.8682285211689921


We can see that limited dictionary improves time significantly, but slightly decreases accuracy.